<a href="https://colab.research.google.com/github/Lorenzo-Gardini/DeepLearnig/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authors
The authors of this code, this notebook and its documentation are [Lorenzo Gardini](mailto: lorenzo.gardini7@studio.unibo.it) and [Vlad Mattiussi](mailto: vlad.mattiussi@studio.unibo.it).

prendi spunto da https://colab.research.google.com/drive/1f9BxFxPxhPKUba8Ifcq4juv042ZA484-

In [ ]:
%autosave 60

In [ ]:
%%capture
!pip install stable-baselines3[extra] ale-py==0.7.4 pyvirtualdisplay tensorflow

In [ ]:
!pip uninstall gym
!pip install gym[atari,accept-rom-license]==0.21.0

# Analisi del problema

il nostro obiettivo è quello di insegnare alla macchina a giocare correttamente al gioco "Pong" per consolle Atari (https://en.wikipedia.org/wiki/Pong). L'obiettivo del gioco è quello di colpire una palla, simulata da un piccolo quadrato, e rispedirla dall'altra parte: il giocatore che non riesce a rispedire la palla concede un punto al suo avversario. Il primo giocatore che raggiunge 21 punti vince la partita.
Per risolvere il problema si propone di ricorrere agli strumenti di reinforcement learning. Quindi si pensa di addestrare un modello basato su alcuni algoritmi. Gli algoritmi utilizzati saranno DQN, DDQN, con experience replay, con prioritized experience replay e Dueling DQN sia di studiare ed utilizzare algoritmi policy based (PPO e A2C).

<img src=https://www.gymlibrary.dev/_images/pong.gif width="300">

**Ambiente d'addestramento:**

Per l'ambiente di addestramento utiliziamo la libreria Gym, la quale ci fornisce il gioco di Pong.
Gym implements the classic “agent-environment loop”: The agent performs some actions in the environment (usually by passing some control inputs to the environment, e.g. torque inputs of motors) and observes how the environment’s state changes. One such action-observation exchange is referred to as a timestep.

<img src=https://www.gymlibrary.dev/_images/AE_loop_dark.png width="500">

The goal in RL is to manipulate the environment in some specific way. For instance, we want the agent to navigate a robot to a specific point in space. If it succeeds in doing this (or makes some progress towards that goal), it will receive a positive reward alongside the observation for this timestep. The reward may also be negative or 0, if the agent did not yet succeed (or did not make any progress). The agent will then be trained to maximize the reward it accumulates over many timesteps.

Spaces are usually used to specify the format of valid actions and observations. Every environment should have the attributes action_space and observation_space, both of which should be instances of classes that inherit from Space. There are multiple Space types available in Gym, nel nostro caso sara' discreto, cioe' describes a discrete space where {0, 1, …, n-1} are the possible values our observation or action can take. Values can be shifted to {a, a+1, …, a+n-1} using an optional argument. Le azioni di Pong saranno 3: su, giu e stare fermi. Per quanto riguarda l'osseravzione by default, the environment returns the RGB image that is displayed to human players as an observation. However, nel nostro caso sara' una immagine in grayscale (Box([[0 ... 0] ... [0  ... 0]], [[255 ... 255] ... [255  ... 255]], (250, 160), uint8)).

Per l'addestramento si è utilizzato la libreria Stable-Baseline con alcune modifiche. Abbiamo modificato il codice della libreria estendendo le classi principali in modo da poter usare gli stessi parametri e struttura della rete usati da deep mind.



## IMPORTS

In [ ]:
import gym
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
from abc import abstractmethod
import random
import time
import random
import statistics

# keras and tensorflow
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.losses import Huber 
from keras.optimizers import Adam, SGD, RMSprop

# save model
import pickle

# video and images manipulation
from IPython.display import display, HTML
from base64 import b64encode
from gym.wrappers import RecordVideo
from stable_baselines3.common.atari_wrappers import AtariWrapper

from skimage import transform
from skimage.color import rgb2gray


## COMMON CONSTANTS


In [ ]:
ENV_NAME = "ALE/Pong-v5"
data_path = "data"

### Display functions

In [ ]:
VIDEO_FOLDER = "./video"

def make_env():
  return gym.make(ENV_NAME)

def play_env(policy, steps):
  env = wrap_video_env(make_env())
  terminated = False
  observation = env.reset()

  for _ in range(steps):
    observation, reward, terminated, _ = env.step(policy(observation))
    if terminated:
      break
  env.close()
  
def wrap_video_env(env):
  return RecordVideo(env, VIDEO_FOLDER)


def show_video():
  mp4 = open(VIDEO_FOLDER + "/rl-video-episode-0.mp4",'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(HTML(f""" <video controls autoplay><source src="%s" type="video/mp4"></video>""" % data_url))

## DQN architecture

Abbiamo utilizzato due modelli diversi di architettura DQN: uno "classico" e uno dueling. Di seguito mostrati in ordine.

The following image shows the architecture of DQN.

<img src=https://biolab.csr.unibo.it/ferrara/Courses/DL/Tutorials/RL/DQN_architecture.png width="1200">

Keras offers a wide range of built-in layers ready for use, including:
- [**Conv2D**](https://keras.io/api/layers/convolution_layers/convolution2d/) - a 2D convolution layer;
- [**Flatten**](https://keras.io/api/layers/reshaping_layers/flatten/) - a simple layer used to flatten the input.

**Dueling DQN**:

This algorithm splits the Q-values in two different parts, the value function V(s) and the advantage function A(s, a).

The value function V(s) tells us how much reward we will collect from state s. And the advantage function A(s, a) tells us how much better one action is compared to the other actions. Combining the value V and the advantage A for each action, we can get the Q-values:

<img src=https://miro.medium.com/max/720/1*81-seZY1rVwC0wzXBprFJg.png width="300" height= "50">

What the Dueling DQN algorithm proposes is that the same neural network splits its last layer in two parts, one of them to estimate the state value function for state s (V(s)) and the other one to estimate the advantage function for each action a (A(s, a)), and at the end it combines both parts into a single output, which will estimate the Q-values. This change is helpful, because sometimes it is unnecessary to know the exact value of each action, so just learning the state-value function can be enough is some cases.

<img src=https://miro.medium.com/max/720/1*vkrLw_sgOgFeBUuZylhyFA.png width="800" height= "400">

However, training the neural network by simply summing the value and advantage functions is not possible. In Q=V+A, given the function Q, we cannot determine the values of V and A, since that is “unidentifiable”. To solve this we can do this: force the highest Q-value to be equal to the value V, thus making the highest value in the advantage function be zero and all other values negative. This will tell us exactly the value for V, and we can calculate all the advantages from there, solving the problem. This is how we would train it:

<img src=https://miro.medium.com/max/720/1*RT7XmyYC8y9uI61tNB1KOQ.png width="500" height= "50">



In [ ]:
def write_data(values, path):
  try:
    with open(path, 'wb') as fp:
      pickle.dump(values, fp, protocol = pickle.HIGHEST_PROTOCOL)
      return True
  except:
    return False

def load_data(path):
  try:
    with open(path, "rb") as parameters:
      return pickle.load(parameters)
  except:
      return {}

In [ ]:
def build_dqn(input_shape=(84, 84, 4), action_count=3):
  model=keras.Sequential(
          [
            layers.Input(shape=input_shape,name='input'),
            layers.Conv2D(filters=32, kernel_size=8, strides=4,activation='relu',padding='valid',name='c1'),
            layers.Conv2D(filters=64, kernel_size=4,strides=2,activation='relu',padding='valid',name='c2'),
            layers.Conv2D(filters=64, kernel_size=3,strides=1,activation='relu',padding='valid',name='c3'),
            layers.Flatten(),
            layers.Dense(512, activation='relu',name='fc'),
            layers.Dense(units=action_count,activation='linear',name='output')
          ]
      )
  model.compile()
  return model

def build_dueling_dqn(input_shape=(84, 84, 4), action_count=3):
  input = layers.Input(shape=input_shape,name='input')
  conv_1 = layers.Conv2D(filters=32, kernel_size=8, strides=4,activation='relu',padding='valid')(input)
  conv_2 = layers.Conv2D(filters=64, kernel_size=4,strides=2,activation='relu',padding='valid')(conv_1)
  conv_3 = layers.Conv2D(filters=64, kernel_size=3,strides=1,activation='relu',padding='valid')(conv_2)
  flatten = layers.Flatten()(conv_3)
  # TODO AGGIUNGI FULLY CONNECTED DOPO FLATTEN

  state = layers.Dense(1,activation='linear',name='state')(flatten)
  raw_advantages = layers.Dense(action_count,activation='linear')(flatten)

  advantages = raw_advantages - tf.reduce_max(raw_advantages, axis=1, keepdims=True)
  Q_values = state + advantages
  model = tf.keras.Model(inputs=[input], outputs=[Q_values])
  model.compile()
  return model

def build_action_target_dqn(input_shape=(84, 84, 4), action_count=3):
  action_model = build_dqn(input_shape, action_count)
  target_model = build_dqn(input_shape, action_count)
  target_model.set_weights(action_model.get_weights())
  return action_model, target_model

def build_dueling_action_target_dqn(input_shape=(84, 84, 4), action_count=3):
  action_model = build_dueling_dqn(input_shape, action_count)
  target_model = build_dueling_dqn(input_shape, action_count)
  target_model.set_weights(action_model.get_weights())
  return action_model, target_model

## Frame preprocessing

To reduce the state complexity, and consequently the computation time, each frame is:
1. transformed in grayscale;
2. cropped to select the region of interest;
3. resized to 84×84.

Moreover, to reduce the memory occupation of the replay buffer, pixel values are stored as bytes (in the range [0;255]) and converted in floating-point values (in the range [0;1]) only when needed as input for the *action* or *target* models.

The following function performs such operations, given:
- the input frame (*frame*); 
- the coordinates of the region of interest (*top_crop*, *bottom_crop*, *left_crop* and *right_crop*);
- the dimension of the resided frame (*resized_shape*).


In [ ]:
TOP_CROP = 14
BOTTOM_CROP = 196
LEFT_CROP = 8
RIGHT_CROP = -1
RESIZE_SHAPE = (84,84)

def preprocess_frame(frame):
  # 1. the input RGB frame is transformed in grayscale
  gray = rgb2gray(frame)
      
  # 2. the region of interest is cropped
  cropped_frame = gray[TOP_CROP:BOTTOM_CROP, LEFT_CROP:RIGHT_CROP]
  
  # 3. the resulting images is resized
  preprocessed_frame = transform.resize(cropped_frame, RESIZE_SHAPE)
  
  # convert float to byte
  byte_preprocessed_frame=(preprocessed_frame*255).astype('uint8')

  return byte_preprocessed_frame

def scale_frames(frames):
  return frames / 255.0

def preprocessed_env():
  return AtariWrapper(make_env())

### Visualize preprocessing

In [ ]:
env = preprocessed_env()
plt.axis('off')
frame = env.reset()
print(frame.squeeze().shape)
plt.imshow(frame.squeeze(), cmap='gray')
plt.show()

NamespaceNotFound: ignored

In [ ]:
env = make_env()
plt.axis('off')
frame = env.reset()
preprocessed_frame = preprocess_frame(frame)
print(preprocessed_frame.shape)
plt.imshow(preprocessed_frame, cmap='gray')
plt.show()

## Frame Stacking


To solve the problem of temporal limitation and give the network the sense of motion, DQN takes a stack of frames as input.

The following function stacks frames together given:
- the new frame to add (*new_frame*);
- the number of frames to stack (*frame_count*);
- the previous frames contained in a **deque** object (*deque_frames*).

For the first *new_frame*, *frame_count* frames identical to *new_frame* are added to a new **deque** object. Otherwise, the *new_frame* is appended to the deque that automatically removes the oldest frame.

In [ ]:
class FramesStack:
  def __init__(self, initial_frame, frame_count, preprocess_frame):
    self._frame_count = frame_count
    self._preprocess_frame = preprocess_frame

    preprocessed_initial_frame = preprocess_frame(initial_frame)
    self._deque = deque([preprocessed_initial_frame for _ in range(frame_count)], maxlen=frame_count)
  
  def add(self, frame):
    self._deque.append( self._preprocess_frame(frame) )
    return self
  
  def stack(self):
    return np.stack(self._deque, axis=2)

## Replay memory

To remove correlations between consecutive transitions and make the DQN training more stable, the *experience replay* technique is used.

The replay memory is implemented as a [**deque**](https://docs.python.org/3/library/collections.html#collections.deque) (Doubly Ended Queue) object providing an O(1) time complexity for append and pop operations from both the ends of the queue. Moreover, if the *maxlen* parameter is specified, the **deque** is bounded to the specified maximum length. Once a bounded length deque is full, when new items are added, a corresponding number of items are discarded from the opposite end.

The DQN authors suggest to populate the replay memory before starting the learning process. For each step $t$, a random action is chosen and executed then the transition $<s_t,a_t,r_{t+1},s_{t+1}>$ is stored in the replay memory. The following function initializes the replay memory given:
- the environment (*env*);
- the replay memory (*replay_memory*);
- the number of transitions to be stored in the replay memory (*replay_memory_init_size*);
- the maximum number of steps per episode (*episode_max_steps*). 


**Prioritized experience replay**:

It is built on top of experience replay buffers, which allow a reinforcement learning (RL) agent to store experiences in the form of transition tuples, usually denoted as (st,at,rt,st+1) with states, actions, rewards, and successor states at some time index t. In contrast to consuming samples online and discarding them thereafter, sampling from the stored experiences means they are less heavily “correlated” and can be re-used for learning.

Uniform sampling from a replay buffer is a good default strategy, and probably the first one to attempt. But prioritized sampling, as the name implies, will weigh the samples so that “important” ones are drawn more frequently for training.

In [ ]:
class ReplayMemory:
  @abstractmethod
  def add_transition(self, transition):
    pass

  def _split_batch(self, batch):
    states, actions, rewards, next_states, dones =  [
            np.array([experience[field_index] for experience in batch])
            for field_index in range(5) 
          ]
    return scale_frames(states), actions, rewards, scale_frames(next_states), dones
  
  @abstractmethod
  def sample_batch(self, batch_size):
    pass

class AbstractReplayMemory(ReplayMemory):
  def __init__(self, max_size):
    self._buffer = deque(maxlen = max_size)

  def len(self):
    return len(self._buffer)

  @abstractmethod
  def add_transition(self, transition):
    pass
  
  @abstractmethod
  def sample_batch(self, batch_size):
    pass

class SimpleReplayMemory(AbstractReplayMemory):
  def __init__(self, max_size):
     super().__init__(max_size)

  def add_transition(self, transition):
    self._buffer.append(transition)
    return self

  def sample_batch(self, batch_size):
    indices = np.random.randint(len(self._buffer), size=batch_size)
    batch = [self._buffer[index] for index in indices]
    return self._split_batch(batch)


class PrioritizedReplayMemory(ReplayMemory):
  @abstractmethod
  def update_priorities(self, indices, losses):
    pass


class PrioritizedReplayMemoryImpl(AbstractReplayMemory, PrioritizedReplayMemory):
  def __init__(self, max_size, alpha = 0.6, error_offset = 0.1):
    super().__init__(max_size)
    self._alpha = alpha
    self._priorities = deque(maxlen = max_size)
    self._default_max_priority = 1
    self._error_offset = 0.1

  def _get_probabilities(self):
    scaled_priorities = np.array(self._priorities) ** self._alpha # pi^a
    sample_probabilities = scaled_priorities/ np.sum(scaled_priorities) # P(i) = pi^a/sum(p^a)
    return sample_probabilities

  def _get_importance(self, probabilities, beta):
    importance = (1/len(self._buffer) * 1/probabilities) ** beta # (1/N * 1/P(i))^b
    normalized_importance = importance / max(importance) # 1/maxi(wi), normalization
    return normalized_importance

  def update_priorities(self, indices, losses):
    for i,l in zip(indices, losses):
      self._priorities[i] = max(abs(l) + self._error_offset, self._default_max_priority)
    
    return self

  def add_transition(self, transaction):
    self._buffer.append(transaction)
    self._priorities.append(max(self._priorities, default = self._default_max_priority))

  def sample_batch(self, batch_size, beta):
    probs = self._get_probabilities()
    sampled_indices = random.choices(range(len(self._buffer)), k=batch_size, weights = probs)
    return (*self._split_batch(np.array(self._buffer)[sampled_indices]), self._get_importance(probs[sampled_indices], beta), sampled_indices)


class TreePrioritizedReplayMemory(PrioritizedReplayMemory):
  def __init__(self, capacity, alpha = 0.6, error_offset = 0.1):
    self._capacity = capacity
    self._alpha = alpha
    self._priority_sum = np.zeros(shape= 2 * capacity, dtype=np.float32)
    self._priority_min = [float('inf') for _ in range( 2 * self._capacity)]
    self._max_priority = 1
    self._data = [None for _ in range(self._capacity)]
    self._next_idx = 0
    self._size = 0
    self._error_offset = error_offset

  def len(self):
    return self._size

  def add_transition(self, transaction):
    idx = self._next_idx
    self._data[idx] = transaction
    self._next_idx = (idx + 1) % self._capacity
    self._size = min(self._capacity, self._size + 1)

    priority_alpha = self._max_priority ** self._alpha
    self._set_priority_min(idx, priority_alpha)
    self._set_priority_sum(idx, priority_alpha)

  def _set_priority_min(self, idx, priority_alpha):
    idx += self._capacity
    self._priority_min[idx] = priority_alpha

    while idx >= 2:
      idx //= 2
      self._priority_min[idx] = min(self._priority_min[2 * idx], self._priority_min[2 * idx + 1])

  def _set_priority_sum(self, idx, priority):
    idx += self._capacity
    self._priority_sum[idx] = priority
    while idx >= 2:
      idx //= 2
      self._priority_sum[idx] = self._priority_sum[2 * idx] + self._priority_sum[2 * idx + 1]

  def _sum(self):
    return self._priority_sum[1]

  def _min(self):
    return self._priority_min[1]

  def _find_prefix_sum_idx(self, prefix_sum):
    idx = 1
    while idx < self._capacity:
        if self._priority_sum[idx * 2] > prefix_sum:
          idx = 2 * idx
        else:
          prefix_sum -= self._priority_sum[idx * 2]
          idx = 2 * idx + 1
    return idx - self._capacity

  def sample_batch(self, batch_size, beta):
    weights = np.zeros(shape=batch_size, dtype=np.float32)
    indices = np.zeros(shape=batch_size, dtype=np.int32)

    # Get sample indices
    for i in range(batch_size):
      p = random.random() * self._sum()
      indices[i] = self._find_prefix_sum_idx(p)

    prob_min = self._min() / self._sum()
    max_weight = (prob_min * self._size) ** (-beta)

    for i in range(batch_size):
      idx = indices[i]
      prob = self._priority_sum[idx + self._capacity] / self._sum()
      weight = (prob * self._size) ** (-beta)
      weights[i] = weight / max_weight

    return (*self._split_batch(self._data[:self._size]), weights, indices)

  def update_priorities(self, indices, losses):
    priorities = np.abs(losses + self._error_offset)

    for idx, priority in zip(indices, priorities):
      self._max_priority = max(self._max_priority, priority)
      priority_alpha = priority ** self._alpha
      self._set_priority_min(idx, priority_alpha)
      self._set_priority_sum(idx, priority_alpha)

  def is_full(self):
    return self._capacity == self._size

In [ ]:
class TreePrioritizedReplayMemory(PrioritizedReplayMemory):
  def __init__(self, capacity, alpha=0.6, error_offset=0.1):
    # We use a power of $2$ for capacity because it simplifies the code and debugging
    self._capacity = capacity
    # $\alpha$
    self._alpha = alpha

    self._error_offset = error_offset

    # Maintain segment binary trees to take sum and find minimum over a range
    self._priority_sum = [0 for _ in range(2 * self._capacity)]
    self._priority_min = np.repeat([float('inf')], 2 * self._capacity)
    self._data = np.zeros(capacity, dtype=object)
    # Current max priority, $p$, to be assigned to new transitions
    self._max_priority = 1.

    # We use cyclic buffers to store data, and `next_idx` keeps the index of the next empty
    # slot
    self._next_idx = 0

    # Size of the buffer
    self._size = 0

  def len(self):
    return self._size

  def add_transition(self, transition):
    # Get next available slot
    idx = self._next_idx

    # store in the queue
    self._data[idx] = transition

    # Increment next available slot
    self._next_idx = (idx + 1) % self._capacity
    # Calculate the size
    self._size = min(self._capacity, self._size + 1)

    # $p_i^\alpha$, new samples get `max_priority`
    priority_alpha = self._max_priority ** self._alpha
    # Update the two segment trees for sum and minimum
    self._set_priority_min(idx, priority_alpha)
    self._set_priority_sum(idx, priority_alpha)

  def _set_priority_min(self, idx, priority_alpha):
    # Leaf of the binary tree
    idx += self._capacity
    self._priority_min[idx] = priority_alpha

    # Update tree, by traversing along ancestors.
    # Continue until the root of the tree.
    while idx >= 2:
      # Get the index of the parent node
      idx //= 2
      # Value of the parent node is the minimum of it's two children
      self._priority_min[idx] = min(self._priority_min[2 * idx], self._priority_min[2 * idx + 1])

  def _set_priority_sum(self, idx, priority):
    # Leaf of the binary tree
    idx += self._capacity
    # Set the priority at the leaf
    self._priority_sum[idx] = priority

    # Update tree, by traversing along ancestors.
    # Continue until the root of the tree.
    while idx >= 2:
      # Get the index of the parent node
      idx //= 2
      # Value of the parent node is the sum of it's two children
      self._priority_sum[idx] = self._priority_sum[2 * idx] + self._priority_sum[2 * idx + 1]

  def _sum(self):
    # The root node keeps the sum of all values
    return self._priority_sum[1]

  def _min(self):
    # The root node keeps the minimum of all values
    return self._priority_min[1]

  def _find_prefix_sum_idx(self, prefix_sum):
    # Start from the root
    idx = 1
    while idx < self._capacity:
      # If the sum of the left branch is higher than required sum
      if self._priority_sum[idx * 2] > prefix_sum:
        # Go to left branch of the tree
        idx = 2 * idx
      else:
        # Otherwise go to right branch and reduce the sum of left
        #  branch from required sum
        prefix_sum -= self._priority_sum[idx * 2]
        idx = 2 * idx + 1

    # We are at the leaf node. Subtract the capacity by the index in the tree
    # to get the index of actual value
    return idx - self._capacity

  def sample_batch(self, batch_size, beta):
    # Initialize samples
    weights = np.zeros(shape=batch_size, dtype=np.float32)
    indices = np.zeros(shape=batch_size, dtype=np.int32)

    # Get sample indices
    for i in range(batch_size):
      p = random.random() * self._sum()
      idx = self._find_prefix_sum_idx(p)
      indices[i] = idx

    # $\min_i P(i) = \frac{\min_i p_i^\alpha}{\sum_k p_k^\alpha}$
    prob_min = self._min() / self._sum()
    # $\max_i w_i = \bigg(\frac{1}{N} \frac{1}{\min_i P(i)}\bigg)^\beta$
    max_weight = (prob_min * self._size) ** (-beta)

    for i in range(batch_size):
      idx = indices[i]
      # $P(i) = \frac{p_i^\alpha}{\sum_k p_k^\alpha}$
      prob = self._priority_sum[idx + self._capacity] / self._sum()
      # $w_i = \bigg(\frac{1}{N} \frac{1}{P(i)}\bigg)^\beta$
      weight = (prob * self._size) ** (-beta)
      # Normalize by $\frac{1}{\max_i w_i}$,
      #  which also cancels off the $\frac{1}{N}$ term
      weights[i] = weight / max_weight

    return (*self._split_batch(self._data[indices]), weights, indices)

  def update_priorities(self, indexes, losses):
    priorities = np.abs(losses + self._error_offset)
    for idx, priority in zip(indexes, priorities):
      # Set current max priority
      self._max_priority = max(self._max_priority, priority)

      # Calculate $p_i^\alpha$
      priority_alpha = priority ** self._alpha
      # Update the trees
      self._set_priority_min(idx, priority_alpha)
      self._set_priority_sum(idx, priority_alpha)

In [ ]:
def dqn_replay_memory_init(env,
                           replay_memory,
                           replay_memory_init_size,
                           episode_max_steps,
                           preprocess_frame,
                           stacked_frame_count):
  loading_step = replay_memory_init_size/10

  while True:
    initial_state = env.reset()
    frames_stack = FramesStack(initial_state, stacked_frame_count, preprocess_frame)
    stacked_states = frames_stack.stack()

    done=False
    step_count=0

    while step_count < episode_max_steps and not done:
      if replay_memory.len() % loading_step == 0:
        print(f'Loaded: {int(replay_memory.len()/replay_memory_init_size * 100)}% ({replay_memory.len()})')

      action = env.action_space.sample()
      new_state, reward, done, _ = env.step(action)

      frames_stack.add(new_state)
      new_stacked_states = frames_stack.stack()

      replay_memory.add_transition([stacked_states, action, reward, new_stacked_states, done])
      stacked_states = new_stacked_states
      step_count+=1
      

      if replay_memory.len() >= replay_memory_init_size:
        return

## Epsilon update

In [ ]:
def update_epsilon(epsilon, min_epsilon, decay):
  return max(min_epsilon,epsilon-decay)

def update_beta(beta, max_beta, beta_increase):
  return min(max_beta, beta+beta_increase)

## Training

To select an action using this DQN, we just pick the action with the largest predicted Q-value. However, to ensure that the agent explores the environment, we choose a random action with probability epsilon.

In [ ]:
def epsilon_greedy_policy(action_model, state, epsilon=0):
    if np.random.rand() < epsilon:
        return env.action_space.sample()
    else:
        Q_values = action_model.predict(state[np.newaxis], verbose=False)
        return Q_values.argmax()  # optimal action according to the DQN

Loop of the game: the agent chooses an action, performs it and the state is updated.

In [ ]:
def game_loop(env, action_model, states, epsilon):
  float_states = scale_frames(states)
  action = epsilon_greedy_policy(action_model, float_states, epsilon)
  next_state, reward, done, _ = env.step(action)
  return action, reward, next_state, done

Given a transition < 𝑠𝑡
, 𝑎𝑡
, 𝑟𝑡+1, 𝑠𝑡+1 >, 𝑄 𝑠𝑡
, 𝑎𝑡 can be expressed by the Bellman
equation in terms of Q-value of next state 𝑠𝑡+1:
𝑄 𝑠𝑡
, 𝑎𝑡 = 𝑟𝑡+1 + 𝛾 ⋅ max
𝑎
𝑄 𝑠𝑡+1.

The maximum future reward for current state 𝑠𝑡 and action 𝑎𝑡
is the immediate reward
𝑟𝑡+1 plus maximum future reward for the next state 𝑠𝑡+1.
 In other words, instead of calculating each value as the sum of the expected cumulative
rewards (which is a long process), this is equivalent to sum the immediate reward and the
discounted future reward of the state that follows.



Since the true value is unknown (RL is unsupervised, and no labels are available), it will be estimated using the Bellman equation:
𝑦𝑡 = 𝑟𝑡+1 + 𝛾 ⋅ max 𝑄 (a) (𝑠𝑡+1,a)


In [ ]:
# target Q values computation
def compute_Q_values(max_next_Q_values, reward_batch, done_batch, gamma):
  target_Q_values = reward_batch + (1 - done_batch) * gamma * max_next_Q_values
  target_Q_values = target_Q_values.reshape(-1, 1)
  return target_Q_values

def target_Q_values(target_model, new_state_batch, reward_batch, done_batch, gamma):
  next_Q_values = target_model.predict(new_state_batch, verbose=0)
  max_next_Q_values = next_Q_values.max(axis=1)
  target_Q_values = compute_Q_values(max_next_Q_values, reward_batch, done_batch, gamma)
  return max_next_Q_values

def double_DQN_values(action_model, target_model, new_state_batch, reward_batch, done_batch, gamma, n_actions):
  next_Q_values = action_model.predict(new_state_batch, verbose=0)  # ≠ target.predict()
  best_next_actions = next_Q_values.argmax(axis=1)
  next_mask = tf.one_hot(best_next_actions, n_actions).numpy()
  max_next_Q_values = (target_model.predict(new_state_batch, verbose=0) * next_mask).sum(axis=1)
  target_Q_values = compute_Q_values(max_next_Q_values, reward_batch, done_batch, gamma)
  return target_Q_values

Q-values can be any real values, which makes it a regression task, that can be optimized
with a simple square error loss.

Perform a gradient descent step to update 𝑄 weights by minimizing the loss function.

To solve the moving target problem, two models (action and target) are used during the DQN training process.

### **Action model update**
The following function updates action model weights using *gradient descent* algorithm given:
- the action model (*dqn_action_model*);
- the target model (*dqn_target_model*);
- a mini-batch containing transitions $<s_i,a_i,r_{i+1},s_{i+1}>$ randomly selected from the replay memory (*mini_batch*); 
- the discount factor $\gamma$ (gamma).

With respect to **simple_dqn_update** function, here the pixel values contained in $s_i$ and $s_{i+1}$ (*state_batch* and *new_state_batch*, respectively) are normalized in the range $[0;1]$ before they being used as input of **predict** and **train_on_batch** methods.

**Double DQN:**

One of the problems of the DQN algorithm is that it overestimates the true rewards; the Q-values think the agent is going to obtain a higher return than what it will obtain in reality. To fix this, the authors of the Double DQN algorithm [1] suggest using a simple trick: decoupling the action selection from the action evaluation. Instead of using the same Bellman equation as in the DQN algorithm, they change it like this:

<img src=https://miro.medium.com/max/720/1*OFYk-9yL6k8QiIUYjjTUXQ.png width="500" height= "50">

First, the main neural network θ decides which one is the best next action a’ among all the available next actions, and then the target neural network evaluates this action to know its Q-value. This simple trick has shown to reduce overestimations, which results in better final policies.

In [ ]:
# generic model update
def update_model(action_model, target_Q_values, state_batch, action_batch, optimizer, loss_fn, n_actions):
  mask = tf.one_hot(action_batch, n_actions)
  with tf.GradientTape() as tape:
    all_Q_values = action_model(state_batch)
    Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
    losses = loss_fn(target_Q_values, Q_values)
    loss = tf.reduce_mean(losses)

  grads = tape.gradient(loss, action_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, action_model.trainable_variables))
  return losses

def prioritized_model_update(action_model, target_model, target_Q_values_fn, batch, gamma, optimizer, loss_fn, n_actions):
  state_batch, action_batch, reward_batch, new_state_batch, done_batch, importances, indices = batch
  target_Q_values = target_Q_values_fn(action_model, target_model, new_state_batch, reward_batch, done_batch, gamma, n_actions)
  new_loss_fn = lambda target_values, predicted_values: loss_fn(target_values, predicted_values) * importances
  losses = update_model(action_model, target_Q_values, state_batch, action_batch, optimizer, new_loss_fn, n_actions)
  return losses, indices

def simple_model_update(action_model, target_model, target_Q_values_fn, batch, gamma, optimizer, loss_fn, n_actions):
  state_batch, action_batch, reward_batch, new_state_batch, done_batch = batch
  target_Q_values = target_Q_values_fn(action_model, target_model, new_state_batch, reward_batch, done_batch, gamma, n_actions)
  update_model(action_model, target_Q_values, state_batch, action_batch, optimizer, loss_fn, n_actions)

In [ ]:
# Double dqn
def double_dqn_prioritized_update(action_model, target_model, batch, gamma, optimizer, loss_fn, n_actions):
  return prioritized_model_update(action_model, target_model, double_DQN_values, batch, gamma, optimizer, loss_fn, n_actions)

def double_dqn_update(action_model, target_model, batch, gamma, optimizer, loss_fn, n_actions):
  return simple_model_update(action_model, target_model, double_DQN_values, batch, gamma, optimizer, loss_fn, n_actions)

# adapter
def target_Q_value_adapter(action_model, target_model, new_state_batch, reward_batch, done_batch, gamma, n_actions):
  return target_Q_values(target_model, new_state_batch, reward_batch, done_batch, gamma)

# Fixed Q
def fixed_Q_prioritized_update(action_model, target_model, batch, gamma, optimizer, loss_fn, n_actions):
  return prioritized_model_update(action_model, target_model, target_Q_value_adapter, batch, gamma, optimizer, loss_fn, n_actions)

def fixed_Q_update(action_model, target_model, batch, gamma, optimizer, loss_fn, n_actions):
  return simple_model_update(action_model, target_model, target_Q_value_adapter, batch, gamma, optimizer, loss_fn, n_actions)

# DQN
def Q_update_prioritized(action_model, batch, gamma, optimizer, loss_fn, n_actions):
  return prioritized_model_update(action_model, action_model, target_Q_value_adapter, batch, gamma, optimizer, loss_fn, n_actions)

def Q_update(action_model, batch, gamma, optimizer, loss_fn, n_actions):
  return simple_model_update(action_model, action_model, target_Q_value_adapter, batch, gamma, optimizer, loss_fn, n_actions)

In [ ]:
def train(env, 
          loss_fn, 
          optimizer,
          mode='DQN', 
          dueling=False,
          episode_count=10_000,
          episode_max_steps=1000,
          replay_memory_max_size=1_000_000,
          replay_memory_init_size=50_000,
          batch_size=32,
          step_per_update=4,
          step_per_update_target_model=500,
          max_epsilon=1,
          min_epsilon=0.1,
          epsilon_decay=1e-06,
          gamma=0.99,
          prioritized_exp_replay=False,
          alpha=0.6,
          max_beta=1,
          min_beta=0.4,
          beta_increase=1e-06,
          preprocess_frame_fn=lambda x:x,
          stacked_frame_count=4,
          moving_avg_window_size = 100,
          moving_avg_stop_thr = None,
          prev_rewards = [],
          episodes_for_saving=20,
          save_data_to=None,
          read_data_from=None):

  #----------------- GLOBAL DECLARATIONS -----------------
  n_actions = env.action_space.n
  epsilon = max_epsilon
  beta = min_beta
  train_rewards = prev_rewards.copy()
  train_step_count = 0 #T
  to_save = {}

  # define the type of replay memory
  if prioritized_exp_replay == True:
    replay_memory = PrioritizedReplayMemoryImpl(replay_memory_max_size, alpha)
  elif prioritized_exp_replay == 'Tree':
    replay_memory = TreePrioritizedReplayMemory(replay_memory_max_size, alpha)
  else:
    replay_memory = SimpleReplayMemory(replay_memory_max_size)

  # define nets
  env_shape = preprocess_frame_fn(env.reset()).shape
 

  # define model structure
  if mode == 'DoubleDQN' or mode == 'FixedDQN':
    if dueling:
      action_model, target_model = build_dueling_action_target_dqn((*env_shape, stacked_frame_count), n_actions)
    else:
      action_model, target_model = build_action_target_dqn((*env_shape, stacked_frame_count), n_actions)
  else: # DQN
    if dueling:
      action_model = build_dueling_dqn((*env_shape, stacked_frame_count), n_actions)
    else:
      action_model = build_dqn((*env_shape, stacked_frame_count), n_actions)

  # define update function
  if mode == 'DoubleDQN':
    if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
      model_update_fn = double_dqn_prioritized_update
    else:
      model_update_fn = double_dqn_update
  elif mode == 'FixedDQN':
    if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
      model_update_fn = fixed_Q_prioritized_update
    else:
      mode_update_fn = fixed_Q_update
  else: # DQN
    if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
      model_update_fn = Q_update_prioritized
    else:
      model_update_fn = Q_update

  # ----------------- READ DATA FROM FILE -----------------
  if read_data_from is not None:
    prev_data = load_data(read_data_from)
    if prev_data:
      train_rewards = prev_data.get('rewards', train_rewards)
      action_model = prev_data.get('action_model', action_model)
      epsilon = prev_data.get('epsilon', epsilon)

      if mode == 'DoubleDQN' or mode == 'FixedDQN':
        target_model = prev_data.get('target_model', target_model)
        
      if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
        beta = prev_data.get('beta', min_beta)
      print(f'Training data restored from file: {read_data_from}')

  # ----------------- INITIALIZATIONS -----------------
  # fill replay memory
  print('Initializing replay memory')
  dqn_replay_memory_init(env, 
                         replay_memory, 
                         replay_memory_init_size, 
                         episode_max_steps, 
                         preprocess_frame_fn, 
                         stacked_frame_count)
  

  # ----------------- START TIMER -----------------
  train_start_time = time.time()

  # ----------------- TRAINING -----------------
  print("training started")
  for n in range(episode_count): 
    #initialization
    # save initial rewards and epsilon (visualization)
    episode_reward = 0
    episode_start_epsilon = epsilon
    # reset the environment and preprocess initial state
    starting_state = env.reset()
    # initialize FramesStack
    frames_deque = FramesStack(starting_state, stacked_frame_count, preprocess_frame_fn)
    stacked_states = frames_deque.stack()
    step_count = 0  #t
    done = False
    
    while step_count < episode_max_steps and not done: 
      # perform a game loop
      action, reward, next_state, done = game_loop(env, action_model, stacked_states, epsilon)
        
      # the stack of frames is updated by adding the new frame and removing the oldest one
      frames_deque.add(next_state)
      new_stacked_states = frames_deque.stack()
      # store transition in the replay memory
      replay_memory.add_transition([stacked_states, action, reward, new_stacked_states, done])
      
      # update the current state
      stacked_states = new_stacked_states

      if train_step_count % step_per_update == 0 and replay_memory.len() >= batch_size:
        # get a random mini-batch from the replay memory
        if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
          mini_batch = replay_memory.sample_batch(batch_size, beta)
        else:
          mini_batch = replay_memory.sample_batch(batch_size)

        # update model
        if mode == 'DoubleDQN' or mode == 'FixedDQN':
          if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
            losses, indices = model_update_fn(action_model, 
                                              target_model, 
                                              mini_batch, 
                                              gamma, 
                                              optimizer, 
                                              loss_fn,
                                              n_actions)
          else:
            model_update_fn(action_model, target_model, mini_batch, gamma, optimizer, loss_fn, n_actions)
        else: # DQN
          if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
            losses, indices = model_update_fn(action_model, 
                                              mini_batch, 
                                              gamma, 
                                              optimizer, 
                                              loss_fn,
                                              n_actions)
          else:
            model_update_fn(action_model, mini_batch, gamma, optimizer, loss_fn, n_actions)

        # update priorities in replay memory 
        if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree': 
          replay_memory.update_priorities(indices, losses)

          
      if train_step_count % step_per_update_target_model == 0 and (mode == 'DoubleDQN' or mode == 'FixedDQN'):
        # copy weights from action to target model
        target_model.set_weights(action_model.get_weights())

      # reduce epsilon
      epsilon = update_epsilon(epsilon, min_epsilon, epsilon_decay)
      beta = update_beta(beta, max_beta, beta_increase)
      
      # increase episode step count and total step count
      step_count += 1
      train_step_count += 1

      # add the current reward to the episode total reward
      episode_reward += reward 
    
    # put the episode total reward into a list for visualization purposes
    train_rewards.append(episode_reward)

    if n > 0 and n % episodes_for_saving == 0 and save_data_to is not None:
      # save model 
      to_save['rewards'] = train_rewards
      to_save['action_model'] = action_model
      to_save['epsilon'] = epsilon

      # add target model
      if mode == 'DoubleDQN' or mode == 'FixedDQN':
        to_save['target_model'] = target_model
      
      if prioritized_exp_replay == True or prioritized_exp_replay == 'Tree':
        to_save['beta'] = beta

      write_data(to_save, save_data_to)
      print('Data saved')

    # for visualization purposes
    
    moving_avg_reward = statistics.mean(train_rewards[-moving_avg_window_size:])
    print(f'Episode: {n+1}° total reward: {episode_reward} [{moving_avg_reward:.2f}]')
    
    # condition to consider the task solved
    if (moving_avg_stop_thr is not None) and moving_avg_reward > moving_avg_stop_thr:
      break

  # --------------- COMPLETE TIME PRINT ---------------
  print(f'Train time: {time.time() - train_start_time}s]')
  # return a list containing the total rewards of all training episodes
  return train_rewards

## Hyper-parameters

In [ ]:
episode_count = 10000                  # Total number of training episodes
episode_max_steps= 1000            # Maximum number of steps per episode

replay_memory_max_size = 1_000_000  # The maximum number of transitions stored into the replay memory. The Deepmind paper suggests 1M however this may cause memory issues.
replay_memory_init_size= 50_000       # The maximum number of transitions stored into the replay memory. The Deepmind paper suggests 50K.
batch_size = 32                     # The mini-batch size

step_per_update = 4                 # The number of total steps executed between successive updates of the action model weights
step_per_update_target_model = 10_000  # The number of total steps executed between successive replaces of the target model weights

max_epsilon=1.0                     # Exploration probability at start
min_epsilon=0.1                     # Minimum exploration probability
epsilon_decay=(max_epsilon-min_epsilon) / 1_000_000.0  # Decay for exploration probability

gamma = 0.99                        # Discount factor

stacked_frame_count= 4               # number of stacked frames

moving_avg_window_size = 100          # Number of consecutive episodes to be considered in the calculation of the total reward moving average
moving_avg_stop_thr = 60              # Minimum value of the total reward moving average to consider the task solved


learning_rate = 0.00025
momentum = 0.95

#prioritized exp replay 
min_beta = 0.4 
max_beta = 1
beta_increase = (max_beta-min_beta) / 1_000_000.0

env = preprocessed_env()
#env = make_env()
loss_fn = Huber()
#optimizer = Adam(learning_rate=learning_rate, clipnorm=1.0)
optimizer = RMSprop(learning_rate = learning_rate, momentum = momentum)

## TRAIN

In [ ]:
train(env, 
      loss_fn,
      optimizer, 
      dueling=False,
      prioritized_exp_replay='Tree',
      epsilon_decay=epsilon_decay,
      beta_increase=beta_increase,
      replay_memory_init_size=1000,
      mode='DQN', 
      save_data_to=data_path,
      read_data_from=data_path,
      preprocess_frame_fn=lambda x : x.squeeze())

# StableBaselines

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import VecVideoRecorder

def make_stable_baselines_env():
  N_ENVS = 1
  FRAMED_STACKED = 4
  SEED = 1234
  env = make_atari_env(ENV_NAME, n_envs=N_ENVS, seed=SEED)
  #env = VecFrameStack(env, n_stack=FRAMED_STACKED)
  return env

def play_env_stable_baselines(model, steps):
  model_name = type(model).__name__
  env = VecVideoRecorder(make_stable_baselines_env(),
                         model_name,
                         record_video_trigger=lambda x: x == 0, 
                         video_length=steps,
                         name_prefix=f"{model_name}")
  observations = env.reset()
  
  for _ in range(steps):
    actions, _ = model.predict(observations)
    observations, rewards, terminateds, _ = env.step(actions)
    if all(terminateds):
      break
  env.close()

In [ ]:
TRAIN_STEPS = 1000
env = make_stable_baselines_env()

def train_model_save_video(model, steps_for_video):
  %tensorboard --logdir log --reload_multifile True
  model.learn(total_timesteps=TRAIN_STEPS)
  model.save("{}_model".format(type(model).__name__))
  play_env_stable_baselines(model, steps_for_video)

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## IMPORTS

In [ ]:
from stable_baselines3.common.atari_wrappers import AtariWrapper, EpisodicLifeEnv, WarpFrame, MaxAndSkipEnv
from stable_baselines3.a2c import A2C
from stable_baselines3.ppo import PPO
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike
from stable_baselines3.common.vec_env import SubprocVecEnv, VecFrameStack, DummyVecEnv, VecMonitor
import gym

## GLOBAL VARIABLES

In [ ]:
timesteps = 120_000_000
frame_stack = 4
frame_skip = 4
ENV_NAME = "ALE/Pong-v5"

## UTILITY FUNCTIONS

In [ ]:
def make_base_env(env_name, frame_skip=4):
  base_env = gym.make(env_name)
  life_env = EpisodicLifeEnv(base_env)
  warp_env = WarpFrame(life_env)
  skip_env = MaxAndSkipEnv(warp_env, frame_skip)
  return skip_env

def make_vec_env(env_name, n_envs=4, frame_stack=4, frame_skip=4, monitor_log=None):
  parallel_vec = SubprocVecEnv([lambda: make_base_env(ENV_NAME, frame_skip) for _ in range(n_envs)])
  stack_vec = VecFrameStack(parallel_vec, n_stack=frame_stack)
  return VecMonitor(stack_vec, monitor_log)

## A2C


Learning a Reinforcement Learning algorithm or a ‘hybrid method’ (A2C) that combines value optimization and policy optimization approaches.

Policy Based agents directly learn a policy (a probability distribution of actions) mapping input states to output actions. Value Based algorithms learn to select actions based on the predicted value of the input state or action.

Both of these methods have considerable drawbacks. That’s why, today, I’ll try another type of Reinforcement Learning method, which we can call a ‘hybrid method’: Actor-Critic. The actor-Critic algorithm is a Reinforcement Learning agent that combines value optimization and policy optimization approaches. More specifically, the Actor-Critic combines the Q-learning and Policy Gradient algorithms. The resulting algorithm obtained at the high level involves a cycle that shares features between:


*   Actor: a PG algorithm that decides on an action to take;
*   Critic: Q-learning algorithm that critiques the action that the Actor selected, providing feedback on how to adjust. It can take advantage of efficiency tricks in Q-learning, such as memory replay.

<img src=https://miro.medium.com/proxy/1*t1rgEDJskqE-iJPg0LI6tw.webp width="300">

The advantage of the Actor-Critic algorithm is that it can solve a broader range of problems than DQN, while it has a lower variance in performance relative to REINFORCE. That said, because of the presence of the PG algorithm within it, the Actor-Critic is still somewhat sampling inefficient.

The actor critic algorithm consists of two networks (the actor and the critic) working together to solve a particular problem. At a high level, the Advantage Function calculates the agent’s TD Error or Prediction Error. The actor network chooses an action at each time step and the critic network evaluates the quality or the Q-value of a given input state. As the critic network learns which states are better or worse, the actor uses this information to teach the agent to seek out good states and avoid bad states.


In my previous tutorial, we derived policy gradients and implemented the REINFORCE algorithm (also known as Monte Carlo policy gradients). There are, however, some issues with vanilla policy gradients: noisy gradients and high variance.


The Advantage Actor Critic has two main variants: the Asynchronous Advantage Actor Critic (A3C) and the Advantage Actor Critic (A2C).

A3C was introduced in Deepmind’s paper “Asynchronous Methods for Deep Reinforcement Learning” (Mnih et al, 2016). In essence, A3C implements parallel training where multiple workers in parallel environments independently update a global value function—hence “asynchronous.” One key benefit of having asynchronous actors is effective and efficient exploration of the state space.

A2C is like A3C but without the asynchronous part; this means a single-worker variant of the A3C. It was empirically found that A2C produces comparable performance to A3C while being more efficient. According to this OpenAI blog post, researchers aren’t completely sure if or how the asynchrony benefits learning:




After reading the paper, AI researchers wondered whether the asynchrony led to improved performance (e.g. “perhaps the added noise would provide some regularization or exploration?“), or if it was just an implementation detail that allowed for faster training with a CPU-based implementation …

Our synchronous A2C implementation performs better than our asynchronous implementations — we have not seen any evidence that the noise introduced by asynchrony provides any performance benefit. This A2C implementation is more cost-effective than A3C when using single-GPU machines, and is faster than a CPU-only A3C implementation when using larger policies.





Anyhow, we will implement the A2C in this post as it is more simple in implementation. (This can easily be extended to A3C)

What’s the advantage function? Considering that “Advantage” is in the Advantage Actor Critic algorithm’s name, it must be pretty important. In order to understand what the Advantage Function is, we first need to understand how to calculate the TD Error, or the Temporal Difference Error.

In Temporal Difference Learning, agents learn by making predictions about future rewards and adjusting their actions based on prediction error. One of the reasons Temporal Difference Learning is quite interesting is that prediction error also seems to be one of the ways that the brain learns new things.

In order to calculate the Advantage Function (TD Error), we need to first calculate the TD Target. In the equation above, the TD Target is the predicted value of all future rewards from the current state S. The function V(s’) represents the Critic Network calculating the value of the next state S’.

<img src=https://miro.medium.com/max/828/0*SCmsjcou4Sr7dkvs width="300">

In the Advantage Actor Critic algorithm, the Advantage is equal to the TD Error shown above. The Advantage can also be interpreted as the Prediction Error of our agent.

<img src=https://miro.medium.com/max/640/0*-ZaG1cRzFH2el5f- width="200">

Note that the advantage function may not always be the same as the TD Error function. For example, in many Policy Gradient algorithms, the advantage is commonly calculated to be the sum of future discounted rewards shown in Figure 4.

The Advantage function tells us if a state is better or worse than expected. If an action is better than expected (the advantage is greater than 0), we want to encourage the actor to take more of that action. If an action is worse than expected (the advantage is less than 0), we want to encourage the actor to take the opposite of that action. If an action performs exactly as expected (the advantage equals 0), the actor doesn’t learn anything from that action.



How does the algorithm decide which actions to encourage and which to discourage? The A2C algorithm makes this decision by calculating the advantage. The advantage decides how to scale the action that the agent just took. Importantly the advantage can also be negative which discourages the selected action. Likewise, a positive advantage would encourage and reinforce that action.

The critic network maps each state to its corresponding Q-value. The Q-value represents the value of a state where Q represents the Quality of the state.
Now that we know how to calculate the TD Target and the TD Error, how do we update the Critic Network weights? Note that as the TD Error approaches 0, the Critic Network gets better and better at predicting the outcome from the current state. In this case, we want to drive the TD Error as close to 0 as possible. In order to update the critic network weights, we use the Mean Squared Error of the TD Error function.

In [ ]:
a2c_n_envs = 16
a2c_monitor_log = 'A2C_monitor'
a2c_model_name = 'A2C_model'

a2c_hyperparams = {
  'ent_coef': 0.01,
  'vf_coef': 0.25,
  'policy_kwargs': dict(optimizer_class=RMSpropTFLike, optimizer_kwargs=dict(eps=1e-5))
}

a2c_env = make_vec_env(ENV_NAME, a2c_n_envs, frame_stack, frame_skip, a2c_monitor_log)
a2c_model = A2C('CnnPolicy', a2c_env, **a2c_hyperparams, verbose=True)
a2c_model.learn(total_timesteps=timesteps, log_interval=1000)
a2c_model.save(a2c_model_name)

## PPO


In the context of RL, a policy π is simply a function that returns a feasible action a given a state s. In policy-based methods, the function (e.g., a neural network) is defined by a set of tunable parameters θ. We can adjust these parameters, observe the differences in resulting rewards, and update θ in the direction that yields higher rewards. This mechanism underlies the notion of all policy gradient methods.

In summary, policy gradients suffers from major drawbacks:



*   Sample inefficiency — Samples are only used once. After that, the policy is updated and the new policy is used to sample another trajectory. As sampling is often expensive, this can be prohibitive. However, after a large policy update, the old samples are simply no longer representative.
*   Inconsistent policy updates — Policy updates tend to overshoot and miss the reward peak, or stall prematurely. Especially in neural network architectures, vanishing and exploding gradients are a severe problem. The algorithm may not recover from a poor update.

*   High reward variance — Policy gradient is a Monte Carlo learning approach, taking into account the full reward trajectory (i.e., a full episode). Such trajectories often suffer from high variance, hampering convergence. [This issue can be addressed by adding a critic, which is outside of the article’s scope]




The idea with Proximal Policy Optimization (PPO) is that we want to improve the training stability of the policy by limiting the change you make to the policy at each training epoch: we want to avoid having too large policy updates.

For two reasons:



*   We know empirically that smaller policy updates during training are more likely to converge to an optimal solution.
*   A too big step in a policy update can result in falling “off the cliff” (getting a bad policy) and having a long time or even no possibility to recover.


So with PPO, we update the policy conservatively. To do so, we need to measure how much the current policy changed compared to the former one using a ratio calculation between the current and former policy. And we clip this ratio in a range [1−ϵ,1+ϵ], meaning that we remove the incentive for the current policy to go too far from the old one (hence the proximal policy term).




The idea was that by taking a gradient ascent step on this function (equivalent to taking gradient descent of the negative of this function), we would push our agent to take actions that lead to higher rewards and avoid harmful actions.

However, the problem comes from the step size:


*   Too small, the training process was too slow
*   Too high, there was too much variability in the training

Here with PPO, the idea is to constrain our policy update with a new objective function called the Clipped surrogate objective function that will constrain the policy change in a small range using a clip.



We update our policy only if:

*   Our ratio is in the range [1−ϵ,1+ϵ]

*   Our ratio is outside the range, but the advantage leads to getting closer to the range 
  *   Being below the ratio but the advantage is > 0
  *   Being above the ratio but the advantage is < 0

You might wonder why, when the minimum is the clipped ratio, the gradient is 0.

To summarize, thanks to this clipped surrogate objective, we restrict the range that the current policy can vary from the old one. Because we remove the incentive for the probability ratio to move outside of the interval since, the clip have the effect to gradient. If the ratio is > 
1+ϵ or < 1−ϵ the gradient will be equal to 0.

The final Clipped Surrogate Objective Loss for PPO Actor-Critic style looks like this, it's a combination of Clipped Surrogate Objective function, Value Loss Function and Entropy bonus:

<img src=https://huggingface.co/blog/assets/93_deep_rl_ppo/ppo-objective.jpg width="500">

Since its introduction in 2017, PPO has quickly established itself as the go-to algorithm in continuous control problems. Five years is a long time in machine learning, yet within its class of benchmark problems (primarily classic control problems and Atari games), it remains highly competitive.

It appears PPO strikes the right balance between speed, caution and usability. Although lacking the theoretical guarantees and mathematical finesse that natural gradients and TRPO do, PPO tends to converge faster and better than its competitors. Somewhat paradoxically, it appears that simplicity pays off in Deep Reinforcement Learning.

In [ ]:
ppo_n_envs = 8
ppo_monitor_log = 'PPO_monitor'
ppo_model_name = 'PPO_model'

ppo_hyperparams = {
  'n_steps': 128,
  'n_epochs': 4,
  'batch_size': 256,
  'learning_rate': 2.5e-4,
  'clip_range': 0.1,
  'vf_coef': 0.5,
  'ent_coef': 0.01,
}

ppo_env = make_vec_env(ENV_NAME, ppo_n_envs, frame_stack, frame_skip, ppo_monitor_log)
ppo_model = PPO('CnnPolicy', ppo_env, **ppo_hyperparams, verbose=True)
ppo_model.learn(total_timesteps=timesteps, log_interval=1000)
ppo_model.save(ppo_model_name)

Using cuda device
Wrapping the env in a VecTransposeImage.
